# Tracks dataset cleaning

In [3]:
import numpy as np
import pandas as pd

# sql alchemy for connecting to database
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import inspect

import os

In [4]:
# read in tracks.csv
df_tracks = pd.read_csv('./tracks.csv')
print(df_tracks.shape)
df_tracks.head(10)

(586672, 20)


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
5,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.227,0.2610,5,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4
6,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,['Francis Marty'],['2nuMRGzeJ5jJEKlfS7rZ0W'],1922,0.510,0.3550,4,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5
7,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,['Mistinguett'],['4AxgXfD7ISvJSTObqm4aIE'],1922,0.563,0.1840,4,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3
8,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,['Greg Fieler'],['5nWlsH5RDgFuRAiDeOFVmf'],1922,0.488,0.4750,0,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4
9,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-29,0.548,0.0391,6,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3


In [5]:
# strip extra brackets and single quotes from artists column
df_tracks['artists'] = df_tracks['artists'].str.strip("['']")

In [6]:
# strip extra brackets and single quotes from id_artists column
df_tracks['id_artists'] = df_tracks['id_artists'].str.strip("['']")

In [7]:
# get first artists only and replace in df
featured_index = df_tracks[df_tracks['artists'].str.contains(',')].index
multiple_artists = df_tracks[df_tracks['artists'].str.contains(',')]
first_artists = multiple_artists['artists'].str.extract(r'((\w*\s)*\w*)\'\,')[0]
df_tracks.loc[df_tracks['artists'].str.contains(','),'artists'] = first_artists

In [8]:
# do the same for artists id 
multiple_id = df_tracks[df_tracks['id_artists'].str.contains(',')]
first_id = df_tracks['id_artists'].str.extract(r'(\w*)\'\,')[0]
df_tracks.loc[df_tracks['id_artists'].str.contains(','),'id_artists'] = first_id

In [9]:
df_tracks.dropna(inplace=True)
df_tracks['release_date'] = df_tracks['release_date'].str.extract(r'(\d{4})')[0]

In [10]:
df_tracks['release_date'] = df_tracks['release_date'].astype(int)

In [11]:
# bin the release_date by decade and place in a list. Append list to dataframe
decade = []
for year in df_tracks['release_date']:
    if (year >= 1940) & (year< 1950):
        decade = "1940's"
    elif (year >= 1950) & (year < 1960):
        decade = "1950's"
    elif (year >= 1960) & (year < 1970):
        decade = "1960's"
    elif (year >= 1970) & (year < 1980):
        decade = "1970's"
    elif (year >= 1980) & (year < 1990):
        decade = "1980's"
    elif (year >= 1990) & (year < 2000):
        decade = "1990's"
    elif (year >= 2000) & (year < 2010):
        decade = "2000's"
    elif (year >= 2010) & (year < 2020):
        decade = "2010's"
    else: 
        decade = "2020's"

# place loop 
df_tracks['decades'] = decade

In [12]:
# drop time_signature ==0 or 1
df_tracks.drop(df_tracks[(df_tracks['time_signature'] == 0) | (df_tracks['time_signature'] == 1)].index, inplace=True)

In [13]:
# reset index and print cleaned dataframe
df_tracks.reset_index(drop=True, inplace=True)
df_tracks.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,decades
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,Uli,45tIt06XoI0Iio4LBEVpls,1922,0.645,0.4450,...,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3,2010's
1,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.434,0.1770,...,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5,2010's
2,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.321,0.0946,...,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3,2010's
3,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.402,0.1580,...,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4,2010's
4,0BRXJHRNGQ3W4v9frnSfhu,Ave Maria,0,178933,0,Dick Haymes,3BiJGZsyX9sJchTqcSA7Su,1922,0.227,0.2610,...,-12.343,1,0.0382,0.994,0.247000,0.0977,0.0539,118.891,4,2010's
5,0Dd9ImXtAtGwsmsAD69KZT,La Butte Rouge,0,134467,0,Francis Marty,2nuMRGzeJ5jJEKlfS7rZ0W,1922,0.510,0.3550,...,-12.833,1,0.1240,0.965,0.000000,0.1550,0.7270,85.754,5,2010's
6,0IA0Hju8CAgYfV1hwhidBH,La Java,0,161427,0,Mistinguett,4AxgXfD7ISvJSTObqm4aIE,1922,0.563,0.1840,...,-13.757,1,0.0512,0.993,0.000016,0.3250,0.6540,133.088,3,2010's
7,0IgI1UCz84pYeVetnl1lGP,Old Fashioned Girl,0,310073,0,Greg Fieler,5nWlsH5RDgFuRAiDeOFVmf,1922,0.488,0.4750,...,-16.222,0,0.0399,0.620,0.006450,0.1070,0.5440,139.952,4,2010's
8,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.548,0.0391,...,-23.228,1,0.1530,0.996,0.933000,0.1480,0.6120,75.595,3,2010's
9,0OYGe21oScKJfanLyM7daU,Capítulo 2.8 - Banquero Anarquista,0,99100,0,Fernando Pessoa,14jtPCOoNZwquk5wd9DxrY,1922,0.676,0.2350,...,-22.447,0,0.9600,0.794,0.000000,0.2100,0.7240,96.777,3,2010's


## Artists dataset cleaning

In [14]:
# read in artists.csv 
file_path = './artists.csv'
df_artists = pd.read_csv(file_path)
df_artists

,id,followers,genres,name,popularity
0,0DheY5irMjBUeLybbCUEZ2,0.0,[],Armid & Amir Zare Pashai feat. Sara Rouzbehani,0
1,0DlhY15l3wsrnlfGio2bjU,5.0,[],ปูนา ภาวิณี,0
2,0DmRESX2JknGPQyO15yxg7,0.0,[],Sadaa,0
3,0DmhnbHjm1qw6NCYPeZNgJ,0.0,[],Tra'gruda,0
4,0Dn11fWM7vHQ3rinvWEl4E,2.0,[],Ioannis Panoutsopoulos,0
...,...,...,...,...,...
1104344,6rJIG42vcWAf1UBdRFlQxB,3345.0,[],Cody Longo,8
1104345,1ljurfXKPlGncNdW3J8zJ8,2123.0,['deep acoustic pop'],Right the Stars,18
1104346,2vnT9YhKIvjVo9LnVjWmr2,26.0,[],Jesse Giddings,0
1104347,3ID0E5XCvnJIYZEq043ZoB,406.0,[],The Boy Band Project,0


In [15]:
# strip the square brackets from the genres column
df_artists['genres'] = df_artists['genres'].str.strip('[]')

In [16]:
# replace empty quotes with null values, then drop null values in genres column and reset index
nan_value = np.nan
df_artists.replace('', nan_value, inplace=True)
df_artists.dropna(subset=['genres'], inplace=True)

# reset index and print cleaned dataframe
df_artists.reset_index(drop=True, inplace=True)
df_artists

,id,followers,genres,name,popularity
0,0VLMVnVbJyJ4oyZs2L3Yl2,71.0,'carnaval cadiz',Las Viudas De Los Bisabuelos,6
1,0dt23bs4w8zx154C5xdVyl,63.0,'carnaval cadiz',Los De Capuchinos,5
2,0pGhoB99qpEJEsBQxgaskQ,64.0,'carnaval cadiz',Los “Pofesionales”,7
3,3HDrX2OtSuXLW5dLR85uN3,53.0,'carnaval cadiz',Los Que No Paran De Rajar,6
4,22mLrN5fkppmuUPsHx6i2G,59.0,"'classical harp', 'harp'",Vera Dulova,3
...,...,...,...,...,...
298611,1q9C5XlekzXbRLIuLCDTre,90087.0,"'social media pop', 'teen pop'",Brent Rivera,33
298612,4fh2BIKYPFvXFsQLhaeVJp,309.0,'la indie',Lone Kodiak,20
298613,7akMsd2vb4xowNTehv3gsY,774.0,'indie rockism',The Str!ke,0
298614,35m7AJrUCtHYHyIUhCzmgi,205.0,'indie rockism',Hunter Fraser,6


In [50]:
# merge df_tracks and df_artists to new dataframe
df_master = pd.merge(df_tracks, df_artists, how='inner', left_on='id_artists', right_on='id', suffixes=('','_drop'))
df_master.head(10)

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,liveness,valence,tempo,time_signature,decades,id_drop,followers,genres,name_drop,popularity_drop
0,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.434,0.1770,...,0.2120,0.457,130.418,5,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
1,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.321,0.0946,...,0.1040,0.397,169.980,3,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
2,0JV4iqw2lSKJaHBQZ0e5zK,Martín Fierro - Remasterizado,0,181173,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.548,0.0391,...,0.1480,0.612,75.595,3,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
3,0l3BQsVJ7F76wlN5QhJzaP,El Vendaval - Remasterizado,0,153533,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.370,0.3720,...,0.9290,0.753,159.669,4,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
4,0xJCJ9XSNcdTIz0QKmhtEn,La Maleva - Remasterizado,0,181440,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.339,0.0958,...,0.1230,0.219,86.279,3,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
5,1HJRj2XztAhgO7dLBw6b7s,El Patotero Sentimental - Remasterizado,0,145627,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.482,0.0283,...,0.0989,0.330,77.642,4,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
6,1KLKkACG16o5crQPiaF1Tz,Tus Suspiros - Remasterizado,0,168107,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.273,0.3180,...,0.7780,0.414,74.542,3,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
7,1KgBLlvPR6BxDyBNDtCUP0,Ojos Funestos - Remasterizado,0,194200,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.373,0.2440,...,0.2140,0.623,115.510,4,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
8,1PGLk4jyXA8oWTBtMyYlwX,El Huerfano - Remasterizado,0,171013,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.462,0.1190,...,0.0773,0.407,122.895,4,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23
9,1VuFIir7DL0mc6bppGMa3r,La Canción del Cabaret - Remasterizado,0,141440,0,Ignacio Corsini,5LiOoJbxVSAMkBS2fUm3X2,1922,0.458,0.1780,...,0.4130,0.286,69.128,5,2010's,5LiOoJbxVSAMkBS2fUm3X2,3528.0,"'tango', 'vintage tango'",Ignacio Corsini,23


In [51]:
# drop duplicate columns in df_master
df_master.drop([col for col in df_master.columns if 'drop' in col], axis=1, inplace=True)
df_master.head(10)

In [55]:
# create engine and connect to the spotify database
engine = create_engine("sqlite:///spotify_db.db")

# map the tables to objects
orm = automap_base()
orm.prepare(engine, reflect=True)
inspector = inspect(engine)

In [56]:
# check for existing tables in the database
inspector.get_table_names()

['artists', 'master', 'tracks']

In [23]:
# create artists table in sqlite database
engine.execute('''
CREATE TABLE artists (
     id VARCHAR(22) NOT NULL,
     followers INT,
	 genres VARCHAR(317),
	 name VARCHAR(194) NOT NULL,
	 popularity INT,
     PRIMARY KEY (id))
     ''')

In [25]:
# create tracks table in sqlite database
engine.execute('''
CREATE TABLE tracks (
	 id VARCHAR(22),
	 name VARCHAR(529),
	 popularity INT,
	 duration_ms INT,
	 explicit INT,
	 artists VARCHAR(100), 
	 id_artists VARCHAR(22),
	 release_date VARCHAR(4),
	 danceability REAL,
	 energy REAL,
	 key INT,
	 loudness REAL,
	 mode INT,
	 speechiness REAL,
	 acousticness REAL,
	 instrumentalness REAL,
	 liveness REAL,
	 valence REAL,
	 tempo REAL,
	 time_signature INT,
	 decades VARCHAR(6),
     PRIMARY KEY (id))
     ''')

In [34]:
# check for artists columns and table schema
columns = inspector.get_columns('artists')
for column in columns:
    print(column)

{'name': 'id', 'type': VARCHAR(length=22), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'followers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'genres', 'type': VARCHAR(length=317), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': VARCHAR(length=194), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [35]:
# check for tracks columns and table schema
columns = inspector.get_columns('tracks')
for column in columns:
    print(column)

{'name': 'id', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'name', 'type': VARCHAR(length=529), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'duration_ms', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'explicit', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'artists', 'type': VARCHAR(length=100), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id_artists', 'type': VARCHAR(length=22), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'release_date', 'type': VARCHAR(length=4), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': '

In [29]:
# write df_tracks to tracks table in spotify database
df_tracks.to_sql('tracks', engine, if_exists='append', index = False)

In [30]:
# write df_artists to artists table in spotify database
df_artists.to_sql('artists', engine, if_exists='append', index = False)

In [53]:
# write df_master to new master table in spotify database
df_master.to_sql('master', engine, if_exists='append', index = False)

In [54]:
# check for master columns and table schema
columns = inspector.get_columns('master')
for column in columns:
    print(column)

{'name': 'id', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'popularity', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'duration_ms', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'explicit', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'artists', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'id_artists', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'release_date', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'danceability', 'type': FLOAT(), 'nullable': True, 'default': 